## 1. MH data load

In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
from PIL import Image

import cv2
import os
import numpy as np

from dataset_MH import *
from googlenet import *

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


In [2]:
# Params
input_size = (512, 512)
batch_size = 16
num_workers = 10

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [3]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb1 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='train', input_size=input_size, augmentation=True)
mhdb2 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='train', input_size=input_size, augmentation=True)
mhdb3 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='train', input_size=input_size, augmentation=True)
mhdb4 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='train', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb1 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='train', input_size=input_size, augmentation=True)
add_mhdb2 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='train', input_size=input_size, augmentation=True)
add_mhdb3 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='train', input_size=input_size, augmentation=True)
add_mhdb4 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='train', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb1 = torch.utils.data.ConcatDataset([mhdb1, add_mhdb1])
concat_mhdb2 = torch.utils.data.ConcatDataset([mhdb2, add_mhdb2])
concat_mhdb3 = torch.utils.data.ConcatDataset([mhdb3, add_mhdb3])
concat_mhdb4 = torch.utils.data.ConcatDataset([mhdb4, add_mhdb4])

In [4]:
mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb1, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb2, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb3, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb4, batch_size=batch_size, shuffle=True, num_workers=num_workers)
#print("mhdb batch size :", len(mhdb_batch2))

## 2. Model

In [5]:
#def __init__(self, in_channel=1, num_classes=1, aux_block=True):
model = GoogLeNet(in_channel=3).cuda()

for i in model.named_children():
    print(i)

('conv_block1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool1', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('conv_block2', Sequential(
  (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('conv_block3', Sequential(
  (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool2', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('inception3a', InceptionBlock(
  (conv_1x1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace)


## 3. Train

In [6]:
# train params
learning_rate = 5e-04
epoch = 5

# optimizer & loss func
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## dataset: mhdb_batch1 / mhdb_batch2 / mhdb_batch3 / mhdb_batch4
#for img, label in mhdb_batch1:
#    print(img, label)

In [7]:
from torch.autograd import Variable

for i in range(epoch):
    model.train(True)
    
    for img, label in mhdb_batch1:
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model(img)
        print(output)
        
        loss = loss_func(output, label)
        loss.backward()
        optimizer.step()

    if i % 1 == 0:
        print(loss)
        

conv_block1 shape :  torch.Size([16, 64, 256, 256])
maxpool2 shape :  torch.Size([16, 192, 64, 64])
maxpool4 shape :  torch.Size([16, 832, 16, 16])
___________________________
(tensor([-0.0614,  0.0082, -0.0302,  0.0078,  0.0013, -0.0194, -0.0041, -0.0064,
        -0.0193, -0.0175, -0.0232, -0.0059,  0.0413, -0.0932,  0.0269, -0.0074],
       device='cuda:0', grad_fn=<AsStridedBackward>), tensor([0.0094, 0.0118, 0.0062, 0.0157, 0.0129, 0.0059, 0.0031, 0.0033, 0.0057,
        0.0160, 0.0134, 0.0320, 0.0161, 0.0090, 0.0213, 0.0035],
       device='cuda:0', grad_fn=<AsStridedBackward>), tensor([0.0172, 0.0274, 0.0215, 0.0192, 0.0217, 0.0200, 0.0308, 0.0308, 0.0121,
        0.0211, 0.0182, 0.0148, 0.0169, 0.0208, 0.0138, 0.0222],
       device='cuda:0', grad_fn=<AsStridedBackward>))


AttributeError: 'tuple' object has no attribute 'log_softmax'